In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw10.ipynb")

# Homework 10: Principal Component Analysis

In lecture we discussed how PCA can be used for dimensionality reduction. Specifically, given a high dimensional dataset, PCA allows us to:
1. Understand the rank of the data. If $k$ principal components capture almost all of the variance, then the data is roughly rank $k$.
2. Create 2D scatterplots of the data. Such plots are a rank 2 representation of our data, and allow us to visually identify clusters of similar observations.

A solid geometric understanding of PCA will help you understand why PCA is able to do these two things. In this homework, we'll build that geometric intuition and look at PCA on two datasets: one where PCA works poorly, and the other where it works pretty well.

## Due Date

This assignment is due **Thursday, November 11th at 11:59 PM PDT**.

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about the homework, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** in the cell below.

**Collaborators:** ...

## Score Breakdown

Question | Points
--- | ---
Question 1a | 1
Question 1b | 1
Question 1c | 1
Question 1d | 1
Question 1e | 1
Question 2a | 2
Question 2b | 1
Question 2c | 1
Question 2d | 3
Question 2e | 2
Question 3a | 1
Question 3b | 1
Question 3c | 1
Question 3d | 2
Question 3e | 2
Question 3f | 2
Question 3g | 1
Question 3h | 2
Question 3i | 2
Total | 28

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px

# Note: If you're having problems with the 3d scatter plots, uncomment the two lines below, and you should see a version that 
#      number that is at least 4.1.1.
# import plotly
# plotly.__version__

## Question 1: PCA on 3D Data

**In question 1, our goal is to see visually how PCA is simply the process of rotating the coordinate axes of our data.**

The code below reads in a 3D dataset. We have named the DataFrame `surfboard` because the data resembles a surfboard when plotted in 3D space.

In [2]:
surfboard = pd.read_csv("data3d.csv")
surfboard.head(5)

The cell below will allow you to view the data as a 3D scatterplot. Rotate the data around and zoom in and out using your trackpad or the controls at the top right of the figure.

You should see that the data is an ellipsoid that looks roughly like a surfboard or a [hashbrown patty](https://www.google.com/search?q=hashbrown+patty&source=lnms&tbm=isch). That is, it is pretty long in one direction, pretty wide in another direction, and relatively thin along its third dimension. We can think of these as the "length", "width", and "thickness" of the surfboard data.

Observe that the surfboard is not aligned with the x/y/z axes.

If you get an error that your browser does not support webgl, you may need to restart your kernel and/or browser.

In [3]:
fig = px.scatter_3d(surfboard, x='x', y='y', z='z', range_x = [-10, 10], range_y = [-10, 10], range_z = [-10, 10])
fig.show()

To give the figure a little more visual pop, the following cell does the same plot, but also assigns a pre-determined color value (that we've arbitrarily chosen) to each point. These colors do not mean anything important, they're simply there as a visual aid.

You might find it useful to use `colorize_surfboard_data` later in this assignment.

In [4]:
def colorize_surfboard_data(df):
    colors = pd.read_csv("surfboard_colors.csv", header = None).values
    df_copy = df.copy()
    df_copy.insert(loc = 3, column = "color", value = colors)
    return df_copy
    
fig = px.scatter_3d(colorize_surfboard_data(surfboard), x='x', y='y', z='z', range_x = [-10, 10], range_y = [-10, 10], range_z = [-10, 10], color = "color", color_continuous_scale = 'RdBu')
fig.show()

## Question 1a

Now that we've understood the data, let's work on understanding what PCA will do when applied to this data.

To properly perform PCA, we will first need to "center" the data so that the mean of each feature is 0. 

Compute the columnwise mean of `surfboard` in the cell below, and store the result in `surfboard_mean`. You can choose to make `surfboard_mean` a numpy array or a series, whichever is more convenient for you. Regardless of what data type you use, `surfboard_mean` should have 3 means, 1 for each attribute, with the x coordinate first, then y, then z.

Then, subtract `surfboard_mean` from `surfboard`, and save the result in `surfboard_centered`. The order of the columns in `surfboard_centered` should be x, then y, then z.

<!--
BEGIN QUESTION
name: q1a
points: 1
-->

In [5]:
surfboard_mean = ...
surfboard_centered = ...

In [ ]:
grader.check("q1a")

## Question 1b

As you may recall from lecture, PCA is a specific application of the singular value decomposition (SVD) for matrices. If we have a data matrix $X$, we can decompose it into $U$, $\Sigma$ and $V^T$ such that $X = U \Sigma V^T$.

In the following cell, use the [`np.linalg.svd`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html) function to compute the SVD of `surfboard_centered`. Store the $U$, $\Sigma$, and $V^T$ matrices in `u`, `s`, and `vt` respectively. This is one line of simple code, exactly like what we saw in lecture.

**Hint:** Set the `full_matrices` argument of `np.linalg.svd` to `False`.

<!--
BEGIN QUESTION
name: q1b
points: 1
-->

In [12]:
...
u, s, vt

In [ ]:
grader.check("q1b")

## Question 1c: Total Variance

Let's now consider the relationship between the singular values `s` and the variance of our data. Recall that the total variance is the sum of the variances of each column of our data. Below, we provide code that computes the variance for each column of the data.

Note: The variances are the same for both `surfboard_centered` and `surfboard`, so we show only one to avoid redundancy.

In [17]:
np.var(surfboard, axis=0)

The total variance of our dataset is given by the sum of these numbers.

In [18]:
total_variance_computed_from_data = sum(np.var(surfboard, axis=0))
total_variance_computed_from_data

As discussed in lecture, the total variance of the data is also equal to the sum of the squares of the singular values divided by the number of data points, that is:

$$Var(X) = \frac{\sum_{i=1}^d{\sigma_i^2}}{N}$$

where $\sigma_i$ is the singular value corresponding to the $i^{th}$ principal component, $N$ is the total number of data points, and $Var(X)$ is the total variance of the data.

In the cell below, compute the total variance using the the formula above and store the result in the variable `total_variance_computed_from_singular_values`. Your result should be very close to `total_variance_computed_from_data`.

<!--
BEGIN QUESTION
name: q1c
points: 1
-->

In [19]:
total_variance_computed_from_singular_values = ...
total_variance_computed_from_singular_values

In [ ]:
grader.check("q1c")

## Question 1d: Explained Variance and Scree Plots

In the cell below, set `variance_explained_by_1st_pc` to the proportion of the total variance explained by the 1st principal component. Your answer should be a number between 0 and 1.

Note: This topic was discussed in [this section of the PCA lecture slides](https://docs.google.com/presentation/d/1zpawVI7o2cYA_C_kSQLBjOMrFkSwMDk23JcedzrzttA/edit#slide=id.ge684cfc9d0_2_98).

<!--
BEGIN QUESTION
name: q1d
points: 1
-->

In [21]:
variance_explained_by_1st_pc = ...
variance_explained_by_1st_pc

In [ ]:
grader.check("q1d")

We can also create a scree plot that shows the proportion of variance explained by all of our principal components, ordered from most to least. An example scree plot is given below. Note that the variance explained by the first principal component matches the value we calculated above for `variance_explained_by_1st_pc`.

Note: If you're wondering where `len(surfboard_centered)` went, it got canceled out when we divided the variance of a given PC by the total variance.

In [24]:
plt.plot([1, 2, 3], s**2 / sum(s**2));
plt.xticks([1, 2, 3], [1, 2, 3]);
plt.xlabel('PC #');
plt.ylabel('Fraction of Variance Explained');
plt.title('Fraction of Variance Explained by each Principal Component')

For this small toy problem, the scree plot is not particularly useful. We'll see why they are useful in practice later in this homework. 

## Question 1e: V as a Rotation Matrix

In lecture, we saw that the first column of $XV$ contained the first principal component values for each observation, the second column of $XV$ contained the second principal component values for each observation, and so forth.

Let's give this matrix a name: $P = XV$ is sometimes known as the "principal component matrix".

Compute the $P$ matrix for the surfboard dataset and store it in the variable `surfboard_pcs`.

<!--
BEGIN QUESTION
name: q1e
points: 1
-->

In [25]:
surfboard_pcs = ...

In [ ]:
grader.check("q1e")

## Visualizing the Principal Component Matrix

In some sense, we can think of $P$ as an output of the PCA procedure. 

It is simply a rotation of the data such that the data will now appear "axis aligned". Specifically, for a 3d dataset, if we plot PC1, PC2, and PC3 along the x, y, and z axes of our plot, then the greatest amount of variation happens along the x-axis, the second greatest amount along the y-axis, and the smallest amount along the z-axis. 

To visualize this, run the cell below, which will show our data now projected onto the principal component space. Compare with your original figure, and observe that the data is exactly the same, only it is now rotated.

In [29]:
surfboard_pcs = surfboard_pcs.rename(columns = {0: "pc1", 1: "pc2", 2: "pc3"})
fig = px.scatter_3d(colorize_surfboard_data(surfboard_pcs), 
                    x='pc1', y='pc2', z='pc3', range_x = [-10, 10], range_y = [-10, 10], range_z = [-10, 10], color = 'color', color_continuous_scale = 'RdBu');
fig.show();

We can also create a 2D scatterplot of our surfboard data as well. Note that the resulting is just the 3D plot as viewed from directly "overhead".

In [30]:
ax = sns.scatterplot(data = colorize_surfboard_data(surfboard_pcs), x = 'pc1', y = 'pc2', hue = "color", palette = "RdBu", legend = False)
ax.set_xlim(-10, 10);
ax.set_ylim(-10, 10);

## Question 1 Summary

Above, we saw that the principal component matrix $P$ is simply the original data rotated in space so that it appears axis-aligned.

Whenever we do a 2D scatter plot of only the first 2 columns of $P$, we are simply looking at the data from "above", i.e. so that the 3rd (or higher) PC is invisible to us.

## Question 2

Let's try out PCA on a higher dimensional dataset. 

In this question, we'll look at a sheet containing Midterm 1 grades from Data 100 in Fall 2019. As it turns out, PCA scatterplots won't be particularly useful on this dataset, but we'll learn some important things along the way.

**Specifically, we'll have 3 primary goals in question 2: we will see what a 2D scatter plot of high dimensional data looks like, we will try to  interpret the meaning of our principal components, and we will see why standardizing columns can be important.**

Note: These grades are a snapshot of the grades before the TAs were done grading. Any problem which hadn't been graded at the time was given a score of 0.

In [31]:
mid1_grades = pd.read_csv("fa19mid1.csv")
mid1_grades.head()

This is 31 dimensional data, with each row of the table representing the scores of a given student. This data has far too many dimensions for us to be able to plot all of the data at once.

One approach is to create a bunch of 2D scatterplots, one for each pair of variables. For example, the cell below generates scatterplots for each of the 5 problems about visualizations.

In [32]:
sns.pairplot(mid1_grades.loc[:, '5.1: 5a i (3.0 pts)':'5.5: 5c (2.0 pts)']);

Unfortunately, due to overplotting, the plot above isn't very informative.

To address overplotting, we can add a little bit of noise. The resulting visualization still isn't great, but we can at least learn a few things, e.g. among students who got 1 point on problem 5b.ii, only one of those students also got 1 point on problem 5a.i.

In [33]:
sns.pairplot(mid1_grades.loc[:, '5.1: 5a i (3.0 pts)':'5.5: 5c (2.0 pts)'] + np.random.normal(0, 0.1, size = (len(mid1_grades), 5)));

## Question 2a

Using PCA, we can try to visualize student performance on ALL questions simultaneously. In the cell below, create a **DataFrame** called `mid1_1st_2_pcs` that has 992 rows and 2 columns, where the first column is named `pc1` and represents the first principal component, and the second column is named `pc2` and represents the second principal component. The columns of your dataframe should be named `pc1` and `pc2`.

**Reminder: make sure to center your data first!**

<!--
BEGIN QUESTION
name: q2a
points: 2
-->

In [34]:
mid1_grades_centered = ...
u_2a, s_2a, vt_2a = np.linalg.svd(mid1_grades_centered, full_matrices = False)
mid1_1st_2_pcs = ...

In [ ]:
grader.check("q2a")

## Question 2b

In the cell below, we create a 2d scatterplot of the first two principal components of the data. Observe that the plot appears to have some sort of structure: The data shows diagonal bands. We will not explore the reasons for these diagonal bands, but we leave this as an optional exercise (Q4) at the very end of the homework if you're curious.

As with the surfboard data, we have assigned arbitrary colors to each student as a visual aid. There is no special meaning to the colors.

In [39]:
def colorize_midterm_data(df):
    """Adds a color column to the given midterm data."""
    colors = pd.read_csv("mid1_colors.csv", header = None).values
    df_copy = df.copy()
    df_copy.insert(loc = 0, column = "color", value = colors)
    return df_copy

In [40]:
sns.scatterplot(data = colorize_midterm_data(mid1_1st_2_pcs), x = "pc1", y = "pc2", hue = "color", palette = "coolwarm", legend = False);

This is a 2D plot of 31 dimensional data. To get a sense of how much of the story we're capturing, in the cell below, give the proportion of the variance accounted for by the first two principal components. Give your answer exactly. It should be around 39%.

<!--
BEGIN QUESTION
name: q2b
points: 1
-->

In [41]:
ds100_mt1_1st_2_pc_variance_fraction = ...
ds100_mt1_1st_2_pc_variance_fraction

In [ ]:
grader.check("q2b")

## Question 2c

While the plot above captures around 39% of the variance, it's hard to interpret. One approach is to do something similar to what we did during lecture where we investigated how much each column of our data contributes to each principal component.

The function defined in the cell below plots and labels the rows of $V^T$.

In [44]:
def plot_pc(col_names, vt, k):
    plt.bar(col_names, vt[k, :], alpha=0.7)
    plt.xticks(col_names, rotation=90);

The cell below plots the first row of $V^T$.

This gives us a chance to try to interpret what PC1 actually means. We see that the first principal component is largely dominated by how a student did on problem 1.7.

Naively, we'd assume that means then that problem 1.7 is the best indicator of student success on the exam. That is, we might expect that 1.7 is a really good problem that deeply tests Data 100 insights.

In [45]:
mid1_col_names = mid1_grades.columns

plt.figure(figsize=(12, 4))
plot_pc(mid1_col_names, vt_2a, 0);

However, if we look at the variance of each individual problem, we see that `problem 1.7: 1e` has a much higher variance than the other problems. After all, PCA is all about identifying the combination of features that best captures variance across observations.

And in case you're curious, this large variance is partially because this problem is worth a lot of points, but also because this snapshot of the grades was at a time when we had only graded a small number of the 1.7 submissions so lots of people still have a zero.

In [46]:
np.var(mid1_grades)

One way to prevent having high variance variables from dominating the first principal component is to rescale our data so that each column has unit variance.

Create a data frame `mid1_grades_centered_scaled` such that the means of each column in `mid1_grades_centered` are 0 and their variances are 1.

*Hint:* Consider how dividing a column by $c$ affects the sample variance of that column. Each column will need to be divided by a different number in order to achieve unit variance in that column.

<!--
BEGIN QUESTION
name: q2c
points: 1
-->

In [47]:
mid1_grades_centered_scaled = ...
mid1_grades_centered_scaled

In [ ]:
grader.check("q2c")

<!-- BEGIN QUESTION -->

## Question 2d

Create a 2D scatterplot of the first two principal components of `mid1_grades_centered_scaled`. Use `colorize_midterm_data` to add a `color` column to `mid1_1st_2_pcs`. Your code will be very similar to the code from problems 2a and 2b. Your result should look like this ![](pca_scatter.png)

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 3
-->

In [51]:
...

<!-- END QUESTION -->



This scatterplot is quite different. The diagonal banding we saw before is gone.

By looking at the colors, we can also get some sense of how our centering process altered the locations of each student in the 2D PCA scatterplot. Observe that the blue students are still mostly close to each other, and the red points are still largely at the margins. 

This plot shows relatively little structure and has no clusters. There does not appear to be much to learn here.

<!-- BEGIN QUESTION -->

## Question 2e

If you compute the fraction of the variance captured by this 2D scatter plot, you'll see it's only 17%, roughly 12% by the 1st PC, and roughly 5% by the 2nd PC. **In the cell below, create a scree plot showing the fraction of the variance explained by each principal componant using the data from 2d.**

Informally, we can say that our midterm scores matrix has a high rank. More formally, we can say that 2 principal components only capture a small fraction of the variance, and thus the data are not particularly amenable to 2D PCA scatterplotting.

<!--
BEGIN QUESTION
name: q2e
manual: true
points: 2
-->

In [52]:
...

<!-- END QUESTION -->



## Question 3

PCA really shines on data where you have reason to believe that the data is relatively low in rank. 

In this final question of the homework, we'll look at how states voted in presidential elections between 1972 and 2016. **Our ultimate goal in question 3 is to show how 2D PCA scatterplots can allow us to identify clusters in a high dimensional dataset.** For this example, that means finding groups of states that vote similarly by plotting their 1st and 2nd principal components.

In [53]:
df = pd.read_csv("presidential_elections.csv")
df.head(5)

The data in this table is pretty messy, so let's create a clean version. The clean table should contain exactly 51 rows (corresponding to the 50 states plus Washington DC) and 13 columns (one for each of the election years from 1972 to 2020). The index of this dataframe should be the state name.

Note: In your personal projects, it is sometimes more convenient to manually do your data cleaning using Excel or Google Sheets. The downside of doing this is that you have no record of what you did, and if you have to redownload the data, you have to redo the manual data cleaning process.

In [54]:
df_clean = ( 
        df.iloc[:, -15:]    
        .drop(['Unnamed: 60'], axis = 1) 
        .rename(columns = {"2000 ‡": "2000", "2016 ‡": "2016", "State.1": "State"}) 
        .drop([51]) 
        .set_index("State")
)
df_clean.head(5)

<!-- BEGIN QUESTION -->

## Question 3a

What does each row in `df_clean` represent?


<!--
BEGIN QUESTION
name: q3a
manual: True
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Question 3b

To perform PCA, we need to convert our data into numerical values. To do this, replace all of the "D" characters with the number 0, and all of the "R" characters with the number 1. 

*Hint:* Use `df.replace`.

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [55]:
df_numerical = ...
...

In [ ]:
grader.check("q3b")

## Question 3c

Now center the data so that the mean of each column is 0 and scale the data so that the variance of each column is 1. Store your result in `df_standardized`.

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [58]:
df_standardized = ...
...

In [ ]:
grader.check("q3c")

## Question 3d

We now have our data in a nice and tidy centered and scaled format, phew. We are now ready to do PCA.

Create a new dataframe `first_2_pcs` that contains exactly the first two columns of the principal components matrix. The first column should be labeled `pc1` and the second column should be labeled `pc2`. Store your result in `first_2_pcs`.

<!--
BEGIN QUESTION
name: q3d
points: 2
-->

In [61]:
u_q3, s_q3, vt_q3 = np.linalg.svd(df_standardized, full_matrices = False)
first_2_pcs = ...
first_2_pcs.head()

In [ ]:
grader.check("q3d")

## Question 3e

The cell below plots the 1st and 2nd principal components of our 50 states + Washington DC.

In [67]:
sns.scatterplot(data = first_2_pcs, x = "pc1", y = "pc2");

<!-- BEGIN QUESTION -->

Unfortunately, we have two problems:

1. There is a lot of overplotting, with only 28 distinct dots. This means that at least some states voted exactly alike in these elections.
2. We don't know which state is which because the points are unlabeled.

Let's start by addressing problem 1. 

**In the cell below, create a new dataframe `first_2_pcs_jittered` with a small amount of random noise added to each principal component. In this same cell, create a scatterplot.**

The amount of noise you add should not significantly affect the appearance of the plot, it should simply serve to separate overlapping observations. Don't get caught up on the exact details of your noise generation, it's fine as long as your plot looks roughly the same as the original scatterplot.

*Hint:* See the pairplot from the intro to question 2 for an example of how to introduce noise.

<!--
BEGIN QUESTION
name: q3e
manual: true
points: 2
-->

In [68]:
...

<!-- END QUESTION -->



## Question 3f

To label our points, the best option is to use the `plotly` library that we used earlier in this homework. `plotly` is an incredibly powerful plotting library that will automatically add axis labels, and will also provide controls so that you can zoom and pan around to look at the data.

One important skill as a user of modern tools is using existing documentation and examples to get the plot you want.

Using the example given on this page as a [guide](https://plot.ly/python/text-and-annotations/), we can create a scatter plot of the **jittered data** from 3e.

In [69]:
import plotly.express as px

state_names = list(df_standardized.index) 
first_2_pcs_jittered['state'] = state_names 

fig = px.scatter(first_2_pcs_jittered, x="pc1", y="pc2", text="state"); 

fig.update_traces(textposition='top center');

fig.show(); 

### Part i

<!-- BEGIN QUESTION -->

Give an example of a cluster of states that vote a similar way. Does the composition of this cluster surprise you? If you're not familiar with U.S. politics, it's fine to just say 'No, I'm not surprised because I don't know anything about U.S. politics.'.

<!--
BEGIN QUESTION
name: q3fi
manual: true
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### Part ii

<!-- BEGIN QUESTION -->

In the cell below, write down anything interesting that you observe by looking at this plot. You will get credit for this as long as you write something reasonable that you can take away from the plot.

<!--
BEGIN QUESTION
name: q3fii
manual: true
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 3g

We can also look at the contributions of each year's elections results on the values for our principal components. Below, we use the `plot_pc` function from question 3c to plot the 1st row of $V^T$ in the cell below.

Here by "1st row" we mean the row that is used to generate `pc1`, and by "2nd row" we mean the row that is used to generate `pc2`.

Note: If you want to adjust the size of your plot for a single figure, you can set `plt.figure(figsize=(x, y))` before creating the figure.

**Note: If you get an error when running this cell, make sure you are properly assigning the `vt_q3` variable in Question 3d.**


In [70]:
plt.figure(figsize=(12, 4))
plot_pc(list(df_standardized.columns), vt_q3, 0);

<!-- BEGIN QUESTION -->

In the cell below, plot the the 2nd row of $V^T$. 

<!--
BEGIN QUESTION
name: q3g
manual: true
points: 1
-->

In [71]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 3h

Using your plots from question 3g as well as the original table, give a description of what it means to have a relatively large positive value for `pc1` (right side of the 2D scatter plot), and what it means to have a relatively large positive value for `pc2` (top side of the 2D scatter plot).

In other words, what is generally true about a state with relatively large positive value for `pc1`? For a large positive value for `pc2`?

Note: `pc2` is pretty hard to interpret, and the staff doesn't really have a consensus on what it means either. We'll be nice when grading. 

Note: Principal components beyond the first are often hard to interpret (but not always; see question 1 earlier in this homework).

<!--
BEGIN QUESTION
name: q3h
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

In [72]:
# feel free to use this cell for scratch work. If you need more scratch space, add cells *below* this one.

# Make sure to put your actual answer in the cell above where it says "Type your answer here, replacing this text"

<!-- BEGIN QUESTION -->

## Question 3i

To get a better sense of whether our 2D scatterplot captures the whole story, create a scree plot for this data. On the y-axis plot the fraction of the total variance captured by the ith principal component. You should see that the first two principal components capture much more of the variance than we were able to capture when using the Data 100 Midterm 1 data. It is partially for this reason that the 2D scatter plot was so much more useful for this dataset.

*Hint:* Your code will be very similar to the scree plot from problem 1d. Be sure to label your axes appropriately!

<!--
BEGIN QUESTION
name: q3i
manual: true
points: 2
-->

In [73]:
...

<!-- END QUESTION -->



## Question 4: Interpreting Diagonal Banding (Optional)

As an optional exercise, try to figure out why we saw the diagonal bands in the scatterplot from question 2b. One approach is to use `plot_pc` on the 1st and 2nd pc to understand how they're created from the data. The answer is somewhat lame, but you might find the exercise useful.

_Type your answer here, replacing this text._

In [74]:
### Feel free to use these cells for scratch work


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()